---------

# Enhanced BiGram model

----------

### Objective is correctness and clarity of concepts and not efficiency

#### Author : Anuj



------
------

IMPROVEMENTS:     
1) Better Tokenization      

LEFTOVERS:   

1) ~~Tensor board graphs for loss~~  

2) Tensor board for network vis      
   (a)   ~~Draw the Plot~~   
   (b)   Interpret and Map the network to Code     

3) Is embedding good or not ?    

4) Embedding visualization ?   
    (a) ~~Plot the vector in an interactive graph~~     
    (b) ~~Plot vectors in Tensorboard~~    
    (c) ~~After every k epochs, take some random words and show their nearest neighbours~~     
    (d) Show vectors being pulled appart in different direction    
    (e) Analogy tasks     
    (f) validation set loss   

5) model perplexity     

6) Input vectors and output vectors     

OPTIMIZATIONS     

1) Better Optimizer     
2) negative sampling    
3) manage learning rate     
4) Param initializations      
     

------
------


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import sys
import tensorflow as tf
import numpy as np
from collections import Counter
import random

In [3]:
sys.path.append("../../data/")


In [4]:
pwd

'/home/piyush/Piyush/study/anthill/Representation-Learning-for-NLP/module2/BiGram'

# Load the data file 

In [5]:
path = "./../../data/harry_potter_3.txt"
text = open(path).read()

raw_data = text.replace('\r\n', ' ')



In [6]:
tokens = raw_data.split()


## frequency counts

In [7]:
# count the frequency of tokens, pick top 100
counts = Counter(tokens)
top_100 = counts.most_common(100)

In [8]:
top_100

[('the', 4578),
 ('to', 2510),
 ('and', 2392),
 ('of', 1990),
 ('a', 1965),
 ('was', 1590),
 ('his', 1401),
 ('said', 1263),
 ('he', 1257),
 ('Harry', 1256),
 ('in', 1112),
 ('had', 865),
 ('at', 761),
 ('you', 717),
 ('--', 710),
 ('I', 706),
 ('it', 684),
 ('that', 667),
 ('on', 621),
 ('with', 620),
 ('as', 590),
 ('for', 487),
 ('He', 464),
 ('out', 427),
 ('Hermione', 402),
 ('Ron', 396),
 ('were', 385),
 ('they', 384),
 ('be', 376),
 ('him', 372),
 ('from', 356),
 ('up', 356),
 ('into', 353),
 ('Professor', 342),
 ('have', 331),
 ('but', 322),
 ('her', 321),
 ('--"', 319),
 ('The', 310),
 ('all', 285),
 ('back', 282),
 ('been', 264),
 ('very', 252),
 ('Harry,', 246),
 ('what', 245),
 ('"I', 241),
 ('who', 240),
 ('their', 237),
 ('looked', 234),
 ('them', 231),
 ('this', 222),
 ('down', 219),
 ('about', 214),
 ('one', 214),
 ('like', 214),
 ('she', 211),
 ('Lupin', 210),
 ('could', 209),
 ('got', 205),
 ('not', 201),
 ('is', 199),
 ('an', 199),
 ('over', 194),
 ('around', 190),
 

## Build token to Id mapping

In [9]:
word_to_id = {}

for token in set(tokens):
    word_to_id[token] = len(word_to_id)


In [10]:
id_to_word = dict(zip(word_to_id.values(), word_to_id.keys()))


# Convert Data to IDs

In [11]:
data = []

data = [word_to_id[tok] for tok in tokens]

In [12]:
print (tokens[:20])

for id in data[:20]:
    print (id_to_word[id]) 

['Harry', 'Potter', 'and', 'the', 'Prisoner', 'of', 'Azkaban', 'by', 'J.K.', 'Rowling', 'CHAPTER', 'ONE', 'OWL', 'POST', 'Harry', 'Potter', 'was', 'a', 'highly', 'unusual']
Harry
Potter
and
the
Prisoner
of
Azkaban
by
J.K.
Rowling
CHAPTER
ONE
OWL
POST
Harry
Potter
was
a
highly
unusual


## Build validation set - randomly choose a batch from these 100

In [13]:
top_100_ids = []

for key, count in top_100:
    top_100_ids.append(word_to_id[key])
    
    
#print top_100_ids

In [14]:
validation_size = 32
validation_set = random.sample(top_100_ids, validation_size)

In [15]:
#top_100_ids

## Now lets slice the data into training set

In [16]:
class BatchData():
    
    def __init__(self, batch_size, list_of_token_ids):
        self.batch_size_ = batch_size
        self.data_ = list_of_token_ids
        self.data_len_ = len(self.data_)
        self.global_pointer = 0
        
    def getBatch(self):
         
        if self.global_pointer + self.batch_size_ > self.data_len_:
            print ("Abort")
            
        X = np.zeros(shape=(self.batch_size_,), dtype=np.int32)
        Y = np.zeros(shape=(self.batch_size_,), dtype=np.int32)
        
        for i in range(self.batch_size_-1):
            
            X[i] = self.data_[self.global_pointer]
            Y[i] = self.data_[self.global_pointer+1]
            
            self.global_pointer = (self.global_pointer + 1)%self.data_len_
            
            
        return X, Y
    
    def reset(self):
        
        self.global_pointer = 0
    

In [17]:
#batch = BatchData(batch_size=32, list_of_token_ids=data)

In [18]:
#X, Y = batch.getBatch()

In [19]:
#print (X)
#print (Y)

In [20]:
#data[:64]

# Network

In [21]:
batch_size=32
num_batches = len(data)/batch_size

embedding_dims = 128
vocab_size = len(set(tokens))

In [22]:
X = tf.placeholder(shape=(batch_size,), dtype=tf.int32, name='X_var')
Y = tf.placeholder(shape=(batch_size,), dtype=tf.int32, name='Y_var')
valid_X = tf.Variable(validation_set, dtype=tf.int32, name='X_valid')

In [23]:
y_oh = tf.one_hot(indices=X, depth=vocab_size, name='Converting_Y_to_Y_oh')

In [24]:
print (X.get_shape())
print (Y.get_shape())
#print (y_oh.get_shape())

(32,)
(32,)


In [25]:
embedding_layer_1 = tf.Variable(tf.truncated_normal(shape=(vocab_size, embedding_dims),mean=0.0, stddev=1.0, dtype=tf.float32), name="Embeddings_Matrix")    
embeded = tf.nn.embedding_lookup(embedding_layer_1, ids=X, name="Embedding_LookUp")

In [26]:
embeded.get_shape()

TensorShape([Dimension(32), Dimension(128)])

In [27]:
#softmax weights, bias
W = tf.Variable(tf.truncated_normal(shape=(embedding_dims, vocab_size),mean=0.0, stddev=1.0, dtype=tf.float32), name="Softmax_Weights_Matrix")
b = tf.Variable(tf.zeros(shape=(vocab_size,)), name="Softmax_Bias_Vector")


In [28]:
logits = tf.add(tf.matmul(embeded, W, name="WX"), b, name="WX_plus_b")

#logits = tf.add(tf.matmul(embed, softmax_weights, name="WX"), softmax_bias, name="WX_plus_b")

loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_oh, name="Compute_Loss")
#mean_loss = tf.reduce_mean(loss)
mean_loss = tf.reduce_mean(loss, name="Compute_mean_loss")

tf.summary.scalar("mean_loss", mean_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<tf.Tensor 'mean_loss:0' shape=() dtype=string>

In [29]:
print (logits.get_shape())
print (y_oh.get_shape())

(32, 15519)
(32, 15519)


In [30]:
optimizer = tf.train.GradientDescentOptimizer(0.5, name="Optimizer").minimize(mean_loss)

In [31]:
summary_op = tf.summary.merge_all()

In [32]:
#compute L2 norm for cosine similarity
norm = tf.sqrt(tf.reduce_sum(tf.square(embedding_layer_1), axis=1, keep_dims=True))
normalised_embeddings = embedding_layer_1 / norm

# get validation set embeddings
validation_data_embeddings = tf.nn.embedding_lookup(normalised_embeddings, ids=valid_X, name="validation_embeddings_lookup")

# similarity score of validation embeddings w.r.t normalised= dot product between validation_data_embeddings and mornalised embeddings
similarity = tf.matmul(validation_data_embeddings, tf.transpose(normalised_embeddings))  # C.A = C x transpose(A)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


# Training

In [36]:
num_of_epochs = 5
LOG_DIR = './bigram_chk_pts'

# A SIMPLE saver() to save the model
saver = tf.train.Saver()

batch = BatchData(batch_size=32, list_of_token_ids=data)

with tf.Session() as sess:
    
    # writer to write graph to tensorboard
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

    tf.global_variables_initializer().run()
    print ("initialised")

    for epoch_id in range(num_of_epochs):

        av_batch_loss = 0

        for batch_id in range(int(num_batches)):

            X_, Y_ = batch.getBatch()

            feed_dict = {}
            feed_dict[X] = X_
            feed_dict[Y] = Y_

            batch_loss, _, summary = sess.run([mean_loss, optimizer,summary_op], feed_dict=feed_dict)
            
            #writer.add_summary(batch_loss, epoch) 
            writer.add_summary(summary, global_step=epoch_id)

            av_batch_loss += batch_loss
            
            if batch_id % 10000 == 0:
                print ("For epoch = %d, batch id = %d, batch loss = %f" %(epoch_id, batch_id, batch_loss))
                
                #print validation data
                sim = similarity.eval() # compute similarity
                
                #iterate over each validation example
                for i in range(validation_size):
                    word = id_to_word[validation_set[i]]
                    top_k = 8
                    # sort indexes and pick top k. we take 1:top_k+1 since 0th top pick will the same word itself
                    nearest = (sim[i,:].argsort()[1:top_k+1])
                    log = '\t Nearest to %s : ' %word
                    for k in range(top_k):
                        nearest_word = id_to_word[nearest[k]]
                        log = '%s %s,' %(log, nearest_word)
                    print (log)
                        
                    
                    

        print ("\nFor epoch = %d, Av loss = %f\n\n" %(epoch_id, av_batch_loss/num_batches))
        
        batch.reset()
        
    save_path = saver.save(sess, LOG_DIR)
    print("Model saved in file: %s" % save_path)
        
        
        
        

initialised
For epoch = 0, batch id = 0, batch loss = 37.268791
	 Nearest to Harry. :  told., "by, emerging, anger., region, Minerva,", no!", He-Who-Must-Not-Be-Named?,
	 Nearest to he :  weeping, doze., sigh., broom., blankly, "SILENCE!, "he, mops,,
	 Nearest to all :  clouding, bred, fragile, collide, openly, lowered, rapidly, booed.,
	 Nearest to is :  occasionally, FAMILY!", outline, trees...., STUPID, confused...., need,, year,",
	 Nearest to from :  mounting, own:, There, might, understood., entangled,, did,", subject.,
	 Nearest to it :  madman,, Harry's., state, Dumbledore's, fireplace., breathlessly, time-out!", feeds,
	 Nearest to but :  seal, Said, aunt., one-armed, trusted., Ravenclaw., minute,, difficulties.,
	 Nearest to think :  were,, do,, amusement, casting, hurriedly., bed.", Now,, done,",
	 Nearest to Lupin :  memory., turbulent, Stringmints,, "Settle, Egypt,, business, Far, tearfully.,
	 Nearest to and :  distance, "YES!", problem?", figure, made, blew, clothes,, "W


For epoch = 2, Av loss = 3.384072


For epoch = 3, batch id = 0, batch loss = 5.726469
	 Nearest to Harry. :  chew,, told., "by, Ernie's., tall,, no!", spellbooks,, emerging,
	 Nearest to he :  weeping, "SILENCE!, "Sit, sister,, possible...., broom., "he, breath,
	 Nearest to all :  me!, conditions!", jobs, surging, first?", clouding, rapidly, collide,
	 Nearest to is :  trees...., unconscious, SCUM!", outline, past, FAMILY!", STUPID, confused....,
	 Nearest to from :  might, mounting, understood., himself), misgivings., injury,", fluttered, twist,
	 Nearest to it :  madman,, "Help, agitated., Harry's., time-out!", fireplace., sweets,, positively,
	 Nearest to but :  aunt., seal, dementors., Said, Goyle's, Ravenclaw., difficulties., trusted.,
	 Nearest to think :  were,, do,, amusement, hurriedly., bed.", "Marvelous,", spoiled, photograph.,
	 Nearest to Lupin :  turbulent, Stringmints,, Egypt,, memory., Far, tearfully., concerned,, rn'dear....",
	 Nearest to and :  figure, clothes,, "

# Plot the Embeddings 

## Tensorboard way

In [38]:
#create metadata file

#reference: https://www.tensorflow.org/get_started/embedding_viz#metadata
LOG_DIR = './bigram_chk_pts'

file_name = LOG_DIR + '/metadata.tsv'
fp = open(file_name, 'w')

for i, v in enumerate(id_to_word):
    print (id_to_word[i])
    
    string = str(i) + '_' + id_to_word[i] + '\n'
    fp.write(string)
    print (string)
    
    #if i > 5:
    #    break
        
fp.close()

streaked
0_streaked

celebrated
1_celebrated

deep,
2_deep,

Firebolt
3_Firebolt

raspberry.
4_raspberry.

gigantic
5_gigantic

delapidated
6_delapidated

Hermione;
7_Hermione;

nowhere,
8_nowhere,

--"
9_--"

pronouncement.
10_pronouncement.

nostrils
11_nostrils

fray;
12_fray;

and
13_and

ashen
14_ashen

classroom.
15_classroom.

wizards!"
16_wizards!"

gap.
17_gap.

repels
18_repels

weekend,"
19_weekend,"

passages
20_passages

ate,
21_ate,

good?"
22_good?"

table;
23_table;

"P
24_"P

good-bye,
25_good-bye,

host
26_host

desert
27_desert

tackling
28_tackling

monsters."
29_monsters."

madness,
30_madness,

"Poor
31_"Poor

transformed,"
32_transformed,"

"Herinione,
33_"Herinione,

animal...
34_animal...

seizure.
35_seizure.

motionless.
36_motionless.

Mice
37_Mice

"'Course
38_"'Course

mouth...
39_mouth...

appeared.
40_appeared.

budding
41_budding

owner
42_owner

easily.
43_easily.

watched,
44_watched,

sideways
45_sideways

secondhand
46_secondhand

Malfoy,
47_Malfoy,


"They'll
1190_"They'll

(Ordinary
1191_(Ordinary

Abandoning
1192_Abandoning

possession
1193_possession

Marge.
1194_Marge.

left,
1195_left,

witty,
1196_witty,

fragments
1197_fragments

punish
1198_punish

pasty
1199_pasty

cords
1200_cords

nonsense!"
1201_nonsense!"

contacting
1202_contacting

James,
1203_James,

theory
1204_theory

tank,
1205_tank,

imprisoned
1206_imprisoned

Trelawneys
1207_Trelawneys

tested,
1208_tested,

syrup
1209_syrup

poor.
1210_poor.

"boos"
1211_"boos"

off;
1212_off;

take?"
1213_take?"

dead!"
1214_dead!"

lengthening,
1215_lengthening,

Muttering
1216_Muttering

Ern!
1217_Ern!

Eve,
1218_Eve,

heroism?
1219_heroism?

scattering
1220_scattering

forever!'
1221_forever!'

note,
1222_note,

take
1223_take

Snape.
1224_Snape.

handed
1225_handed

squelch
1226_squelch

loss
1227_loss

Prongs
1228_Prongs

dead."
1229_dead."

fury
1230_fury

WE
1231_WE

class's
1232_class's

clean.
1233_clean.

wafting
1234_wafting

Scabbers."
1235_Scabbers."

else!"
12

2439_buns

responsible
2440_responsible

unenthusiastic
2441_unenthusiastic

floor?"
2442_floor?"

OWN.
2443_OWN.

cafes,
2444_cafes,

outline
2445_outline

angrily.
2446_angrily.

knees
2447_knees

Nimbus
2448_Nimbus

"Yeh
2449_"Yeh

hero's
2450_hero's

refilled
2451_refilled

straightening
2452_straightening

shake.
2453_shake.

hood,
2454_hood,

traditional
2455_traditional

glances
2456_glances

this
2457_this

delicately
2458_delicately

murdering
2459_murdering

Hall,
2460_Hall,

shrill,
2461_shrill,

now...."
2462_now...."

crashed
2463_crashed

ever,
2464_ever,

FOR
2465_FOR

like...
2466_like...

having,
2467_having,

repossessed
2468_repossessed

affair
2469_affair

feverishly.
2470_feverishly.

convulsively
2471_convulsively

locks
2472_locks

air....
2473_air....

stroking
2474_stroking

-
2475_-

match!"
2476_match!"

aside
2477_aside

twice!
2478_twice!

aboard...."
2479_aboard...."

broomstick?"
2480_broomstick?"

repair,
2481_repair,

double-ended
2482_double-ended

any

can't
3465_can't

called,
3466_called,

Firebolt?
3467_Firebolt?

splattering
3468_splattering

leaves,
3469_leaves,

smuggled
3470_smuggled

Remus...."
3471_Remus...."

"They're
3472_"They're

as
3473_as

parents'
3474_parents'

dreams
3475_dreams

whiled
3476_whiled

smacking
3477_smacking

one's
3478_one's

Shunpike,
3479_Shunpike,

stormy
3480_stormy

superiority.
3481_superiority.

eh?
3482_eh?

wondered
3483_wondered

Potter,
3484_Potter,

plummeted
3485_plummeted

exhausted
3486_exhausted

off-color
3487_off-color

tadpole.
3488_tadpole.

listen
3489_listen

first!
3490_first!

essay..."
3491_essay..."

delight
3492_delight

squeaking.
3493_squeaking.

poured
3494_poured

things."
3495_things."

absently
3496_absently

reasoned,
3497_reasoned,

"JORDAN!
3498_"JORDAN!

argued,
3499_argued,

pub
3500_pub

neck,
3501_neck,

Kep'
3502_Kep'

unlike
3503_unlike

sometimes
3504_sometimes

"What!
3505_"What!

delicate!"
3506_delicate!"

Tower,"
3507_Tower,"

crack.
3508_crack.

Ad
3509_

(two
4689_(two

O1iver
4690_O1iver

drew
4691_drew

Gotta
4692_Gotta

me
4693_me

teacher.
4694_teacher.

laugh,
4695_laugh,

yourself!"
4696_yourself!"

tried,"
4697_tried,"

eyelids
4698_eyelids

package,
4699_package,

own
4700_own

understannd
4701_understannd

leg
4702_leg

aching
4703_aching

Pettigrew's
4704_Pettigrew's

lakeshore,
4705_lakeshore,

ages,
4706_ages,

distance
4707_distance

exit,
4708_exit,

kind
4709_kind

MAP
4710_MAP

clench.
4711_clench.

short
4712_short

frequent
4713_frequent

lady!
4714_lady!

still,
4715_still,

"We
4716_"We

bulldogs.
4717_bulldogs.

extremely
4718_extremely

Surely
4719_Surely

UP
4720_UP

encourage
4721_encourage

snap,
4722_snap,

himself,"
4723_himself,"

Shop,
4724_Shop,

BANG!
4725_BANG!

speaking."
4726_speaking."

toothless
4727_toothless

Bludger
4728_Bludger

coward,
4729_coward,

started
4730_started

tail
4731_tail

mum
4732_mum

party
4733_party

boos
4734_boos

nutters,
4735_nutters,

retaliation.
4736_retaliation.

uncle


...

moment;
9188_moment;

Potters
9189_Potters

same
9190_same

Harry;
9191_Harry;

around."
9192_around."

Need
9193_Need

family.
9194_family.

rage
9195_rage

Lupin."
9196_Lupin."

male
9197_male

forest.
9198_forest.

died...
9199_died...

free,"
9200_free,"

Under
9201_Under

AN,
9202_AN,

mumbled.
9203_mumbled.

work,"
9204_work,"

fully
9205_fully

family
9206_family

password!"
9207_password!"

lives
9208_lives

today?"
9209_today?"

Mooney
9210_Mooney

attempts
9211_attempts

abouts
9212_abouts

knocked,
9213_knocked,

guards
9214_guards

"He'd've
9215_"He'd've

February
9216_February

positioned
9217_positioned

"Looks
9218_"Looks

FREE
9219_FREE

"Dissendium!"
9220_"Dissendium!"

tower?
9221_tower?

ladder
9222_ladder

My
9223_My

poisoned
9224_poisoned

highpitched
9225_highpitched

essays,
9226_essays,

whirl
9227_whirl

thinking
9228_thinking

tinkle
9229_tinkle

speed;
9230_speed;

sprinted
9231_sprinted

catch
9232_catch

tie
9233_tie

bush
9234_bush

facedown
9235_facedown


insist."
10438_insist."

dash
10439_dash

"this
10440_"this

players);
10441_players);

Bye
10442_Bye

life-threatening,"
10443_life-threatening,"

although
10444_although

fled
10445_fled

insisted
10446_insisted

bag.
10447_bag.

coat
10448_coat

true?"
10449_true?"

framed
10450_framed

pit
10451_pit

ragged
10452_ragged

murmured
10453_murmured

unregistered
10454_unregistered

died,
10455_died,

recovered;
10456_recovered;

"Eh?"
10457_"Eh?"

visits,
10458_visits,

"Dead,"
10459_"Dead,"

Championship
10460_Championship

ferreting
10461_ferreting

father
10462_father

destroyed,
10463_destroyed,

lights.
10464_lights.

snowflakes,
10465_snowflakes,

evening
10466_evening

gasped.
10467_gasped.

mine."
10468_mine."

Sprout
10469_Sprout

Fang?"
10470_Fang?"

skulked
10471_skulked

Orb,
10472_Orb,

musta
10473_musta

stopping,
10474_stopping,

rest.
10475_rest.

"you've
10476_"you've

Hooch?
10477_Hooch?

group.
10478_group.

"Unfogging
10479_"Unfogging

standard,
10480_standard,

li

11687_teacup.

members
11688_members

"Ron,
11689_"Ron,

examination
11690_examination

explode
11691_explode

PASSING
11692_PASSING

Tall-Twig
11693_Tall-Twig

album
11694_album

branches.
11695_branches.

reminds
11696_reminds

bequeath
11697_bequeath

ah
11698_ah

Lunatic
11699_Lunatic

rise.
11700_rise.

lighting
11701_lighting

whip
11702_whip

'mall
11703_'mall

Twenty
11704_Twenty

Deciding
11705_Deciding

of!"
11706_of!"

straight....
11707_straight....

lonely.
11708_lonely.

One.
11709_One.

Azkaban.
11710_Azkaban.

Very
11711_Very

reason
11712_reason

'ad
11713_'ad

arrived
11714_arrived

returned,
11715_returned,

too!
11716_too!

rocking
11717_rocking

cramming
11718_cramming

grave
11719_grave

whining:
11720_whining:

packing?"
11721_packing?"

Sirius!
11722_Sirius!

national
11723_national

turns
11724_turns

adult
11725_adult

recovery.
11726_recovery.

loony,
11727_loony,

caused,
11728_caused,

peace,
11729_peace,

scarves,
11730_scarves,

anger
11731_anger

please,

minute,
13187_minute,

school,
13188_school,

sliding
13189_sliding

hardest
13190_hardest

upset,"
13191_upset,"

Sunday
13192_Sunday

floor.
13193_floor.

compiled
13194_compiled

results
13195_results

birthdays'
13196_birthdays'

Further
13197_Further

Eye."
13198_Eye."

fulfill
13199_fulfill

"Like
13200_"Like

packed
13201_packed

need's
13202_need's

"I'd
13203_"I'd

revive
13204_revive

"Pettigrew...
13205_"Pettigrew...

GRUDGE
13206_GRUDGE

fancied.
13207_fancied.

hesitate
13208_hesitate

"So,"
13209_"So,"

calmed
13210_calmed

""as
13211_""as

double
13212_double

wood
13213_wood

string.
13214_string.

whoever
13215_whoever

"Can'
13216_"Can'

slimeball."
13217_slimeball."

Three
13218_Three

Ginny.
13219_Ginny.

chatter
13220_chatter

HARRY
13221_HARRY

"Shh,
13222_"Shh,

thoughtfully.
13223_thoughtfully.

practicing
13224_practicing

everywhere.
13225_everywhere.

top
13226_top

squeaky.
13227_squeaky.

grindylows,"
13228_grindylows,"

patch.
13229_patch.

explaining.
132

form!
14437_form!

Lupin!"
14438_Lupin!"

dear?
14439_dear?

alarming.
14440_alarming.

throat,
14441_throat,

dessert
14442_dessert

brain....
14443_brain....

lowerng
14444_lowerng

However,
14445_However,

glittering,
14446_glittering,

THREE
14447_THREE

"Eat,"
14448_"Eat,"

widely
14449_widely

beckoned
14450_beckoned

Honeydukes
14451_Honeydukes

bike
14452_bike

explains
14453_explains

talentless
14454_talentless

Professor."
14455_Professor."

wheels.
14456_wheels.

really,
14457_really,

position,
14458_position,

exam,
14459_exam,

Behind
14460_Behind

rise;
14461_rise;

notes.
14462_notes.

SCABBERS!"
14463_SCABBERS!"

pushing
14464_pushing

energy
14465_energy

boomed
14466_boomed

rats,
14467_rats,

practice."
14468_practice."

desperate
14469_desperate

point.
14470_point.

humans
14471_humans

wands;
14472_wands;

of-term
14473_of-term

looked,
14474_looked,

frozen.
14475_frozen.

that's
14476_that's

energy.
14477_energy.

gathered
14478_gathered

though,
14479_though

In [39]:
from tensorflow.contrib.tensorboard.plugins import projector
import os

summary_writer = tf.summary.FileWriter(LOG_DIR)

#get config params
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embedding_layer_1.name  # this is where you connect your embedding tensor

# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)

# run your tensorboard
# tensorboard --logdir="./bigram_chk_pts" --port 6006